In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import datetime
import re

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

/Users/1aguschin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/1aguschin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Задание

С помощью генерации признаков получить rmse меньше 5000 на тестовой части выборки.

Постарайтесь получить такой результат сначала с помощью **RandomForestRegressor**, а затем и с помощью **Ridge**.

Если с помощью признаков меньше 5000 не получается, можно попробовать улучшить результат с помощью обучения других моделей, подбора параметров и усреднения предсказаний разных моделей.

Для генерации признаков вам сможет помочь код из последнего семинара.

# предобработаем данные

In [2]:
data = pd.read_csv('MotorcycleData.csv', encoding='cp1252', sep=',', )
data.drop('Unnamed: 0', axis=1, inplace=True)

f = lambda x: re.sub('[,$]', '', x)
data.Price = data.Price.map(f).astype(int)

data.head(2)

,Condition,Condition_Desc,Price,Location,Model_Year,Mileage,Exterior_Color,Make,Warranty,Model,...,Vehicle_Title,OBO,Feedback_Perc,Watch_Count,N_Reviews,Seller_Status,Vehicle_Tile,Auction,Buy_Now,Bid_Count
0,Used,mint!!! very low miles,11412,"McHenry, Illinois, United States",2013.0,"16,000",Black,Harley-Davidson,Unspecified,Touring,...,NaN,False,8.1,NaN,2427,Private Seller,Clear,True,False,28.0
1,Used,Perfect condition,17200,"Fort Recovery, Ohio, United States",2016.0,60,Black,Harley-Davidson,Vehicle has an existing warranty,Touring,...,NaN,False,100,17,657,Private Seller,Clear,True,True,0.0


# разобьем их на трейн и тест

часть выборки мы сохраним и будем использовать как тестовую выборку

In [3]:
target = 'Price'

skf = KFold(shuffle=True, random_state=0)
for itr, ite in skf.split(data):
    break

y_train = data.loc[itr, target].values
y_test = data.loc[ite, target].values
data.loc[ite, target] = np.nan

print(len(itr), len(ite))

4995 2498


# генерация признаков

Основные изменения в коде должны произойти в этом блоке.

**Краткое напоминание** :

0. Числовые признаки
    - для линейных моделей признаки стоит масштабировать
1. Категориальные признаки
    - для линейных моделей лучше сделать one-hot-encoding
    - для деревьев лучше сделать LabelEncoding
    - для категориальных признаков высокой размерности (>10) можно сделать кодирование средним значением таргета. Это облегчит работу и для деревьев, и для линейных моделей
2. Время и координаты
    - оба типа признаков обладают некоторым внутренним порядком (два момента времени можно сравнить, широту/долготу тоже)
    - оба типа признаков позволяют посчитать расстояния между объектами

In [4]:
xcols = [c for c in data if c != target]
print(xcols)

['Condition', 'Condition_Desc', 'Location', 'Model_Year', 'Mileage', 'Exterior_Color', 'Make', 'Warranty', 'Model', 'Sub_Model', 'Type', 'Vehicle_Title', 'OBO', 'Feedback_Perc', 'Watch_Count', 'N_Reviews', 'Seller_Status', 'Vehicle_Tile', 'Auction', 'Buy_Now', 'Bid_Count']


In [5]:
X = data[xcols].select_dtypes(['int', 'float'])
X = X.fillna(0)
X[:1]

,Model_Year,Vehicle_Title,N_Reviews,Bid_Count
0,2013.0,0.0,2427,28.0


# валидация

In [6]:
X_train = X.loc[itr]
X_test  = X.loc[ite]

In [7]:
model = RandomForestRegressor()
# model = Ridge()

score_mse = -cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error')
score_rmse = score_mse ** 0.5
print(np.mean(score_rmse))

9072.065548066726


# проверка на тестовых данных
rmse на тестовой выборке должно получиться меньше 5000

In [8]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

def rmse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2) ** 0.5

rmse(y_test, y_pred)

7407.8182220921035